In [2]:
import torch
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from transformers import DataCollatorWithPadding
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import PyTorchBenchmark, PyTorchBenchmarkArguments

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -q py3nvml

In [3]:
cedr = load_dataset('cedr')

No config specified, defaulting to: cedr/main
Reusing dataset cedr (/root/.cache/huggingface/datasets/cedr/main/0.1.1/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def binarize_labels(labels):
    return [int(len(labels)==0) ] + [int(i in labels) for i in range(5)]

In [5]:
MODEL_NAME = 'cointegrated/rubert-tiny2-cedr-emotion-detection'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=6, 
    problem_type='multi_label_classification'
    )

In [6]:
cedr_mapped = cedr.map(
    lambda x: tokenizer(
        x['text'],
        truncation=True), 
    batched=True
    ).map(
        lambda x: {'label': [float(y) for y in binarize_labels(x['labels'])]},
        batched=False,
        remove_columns=['text', 'labels', 'source']
        )

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/7528 [00:00<?, ?ex/s]

  0%|          | 0/1882 [00:00<?, ?ex/s]

In [7]:
data_collator = DataCollatorWithPadding(tokenizer)

In [8]:
def predict_with_model(model, dataloader):
    preds = []
    facts = []

    for batch in tqdm(dataloader):
        facts.append(batch.labels.cpu().numpy())
        batch = batch.to(model.device)
        
        with torch.no_grad():
            pr = model(
                input_ids=batch.input_ids,
                attention_mask=batch.attention_mask,
                token_type_ids=batch.token_type_ids
                )
        preds.append(torch.softmax(pr.logits, -1).cpu().numpy())

    facts = np.concatenate(facts)
    preds = np.concatenate(preds)

    return facts, preds


def get_classification_report(facts, preds, model):
    aucs = {label:roc_auc_score(facts[:, i], preds[:, i]) for i, label in model.config.id2label.items()}
    return aucs


def evaluate_model(model, dev_dataloader):
    facts, preds = predict_with_model(model, dev_dataloader)
    aucs = get_classification_report(facts, preds, model)
    aucs['overall'] = np.mean(list(aucs.values()))
    return aucs

In [9]:
batch_size = 64

In [10]:
test_dataloader = DataLoader(
    cedr_mapped['test'], 
    batch_size=batch_size,
    drop_last=False,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator
)

In [11]:
test_results = evaluate_model(model, test_dataloader)

  0%|          | 0/30 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [12]:
test_results

{'no_emotion': 0.9285743954656361,
 'joy': 0.9511928957992504,
 'sadness': 0.956353783198774,
 'surprise': 0.8908363111599781,
 'fear': 0.8954949670239243,
 'anger': 0.7511348890153671,
 'overall': 0.8955978736104884}

In [13]:
benchmark_args = PyTorchBenchmarkArguments(
    models=[MODEL_NAME],
    training=False,
    inference=True,
    sequence_lengths=[8,64,128,256,512],
    batch_sizes=[1,8,32,64],
    multi_process=True,
    cuda=True,
    speed=True,
)

benchmark = PyTorchBenchmark(benchmark_args)

In [ ]:
results = benchmark.run()

In [16]:
results

BenchmarkOutput(time_inference_result={'cointegrated/rubert-tiny2-cedr-emotion-detection': {'bs': [1, 8, 32, 64], 'ss': [8, 64, 128, 256, 512], 'result': {1: {8: 0.0029936344013549387, 64: 0.00341445910744369, 128: 0.003241317404899746, 256: 0.00455097509548068, 512: 0.003569496003910899}, 8: {8: 0.003511209797579795, 64: 0.003955621505156159, 128: 0.003162354801315814, 256: 0.006981807597912848, 512: 0.01921859800349921}, 32: {8: 0.003725462395232171, 64: 0.004535592696629465, 128: 0.009932486701291054, 256: 0.025330946804024278, 512: 0.07335092639550567}, 64: {8: 0.0031430474016815426, 64: 0.008397517004050314, 128: 0.01852042810060084, 256: 0.04917940800078213, 512: 0.1455255846027285}}}}, memory_inference_result={'cointegrated/rubert-tiny2-cedr-emotion-detection': {'bs': [1, 8, 32, 64], 'ss': [8, 64, 128, 256, 512], 'result': {1: {8: 805, 64: 805, 128: 805, 256: 809, 512: 821}, 8: {8: 805, 64: 809, 128: 825, 256: 853, 512: 1017}, 32: {8: 809, 64: 825, 128: 897, 256: 1057, 512: 1713